In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os
import imageio
%rm -rf frames

def parse_ttp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    coordinates_section = False
    items_section = False
    cities_coordinates = []
    values = []
    weights = []
    item_assignments = []
    for line in lines:
        if line.startswith('NODE_COORD_SECTION'):
            coordinates_section = True
            continue
        if line.startswith('ITEMS SECTION'):
            coordinates_section = False
            items_section = True
            continue
        if coordinates_section:
            parts = line.strip().split()
            if len(parts) == 3:
                cities_coordinates.append([float(parts[1]), float(parts[2])])
        if items_section:
            parts = line.strip().split()
            if len(parts) == 4:
                values.append(int(parts[1]))
                weights.append(int(parts[2]))
                item_assignments.append(int(parts[3]) - 1)
    cities_coordinates = np.array(cities_coordinates)
    values = np.array(values)
    weights = np.array(weights)
    item_assignments = np.array(item_assignments)
    return cities_coordinates, values, weights, item_assignments

file_path = 'eil51_01_bsc_01_02.thop'
cities_coordinates, values, weights, item_assignments = parse_ttp_file(file_path)
max_weight = 4029
speed_vmax = 1.00
speed_vmin = 0.10
num_ants = 10
num_cities = cities_coordinates.shape[0]
num_items = values.shape[0]
time_limit = 300
distances = np.linalg.norm(cities_coordinates[:, np.newaxis] - cities_coordinates, axis=2)
pheromone_matrix = np.ones((num_cities, num_cities))

class Ant:
    def __init__(self, num_cities, start_city, end_city):
        self.num_cities = num_cities
        self.start_city = start_city
        self.end_city = end_city
        self.tour = [start_city] + list(np.random.permutation(range(1, num_cities))) + [end_city]
        self.total_value = 0
        self.total_weight = 0
        self.tour_distance = self.calculate_tour_distance()
        self.tour_time = self.calculate_tour_time()

    def calculate_tour_distance(self, tour=None):
        if tour is None:
            tour = self.tour
        tour_distance = 0.0
        for i in range(len(tour) - 1):
            tour_distance += distances[tour[i], tour[i + 1]]
        return tour_distance

    def calculate_tour_time(self):
        tour_time = 0.0
        for i in range(len(self.tour) - 1):
            current_speed = max(speed_vmin, speed_vmax - (self.total_weight * (speed_vmax - speed_vmin) / max_weight))
            tour_time += distances[self.tour[i], self.tour[i + 1]] / current_speed
        return tour_time

    def update_solution(self, probabilities):
        unvisited_cities = set(range(self.num_cities)) - {self.start_city, self.end_city}
        self.tour = [self.start_city]
        current_city = self.start_city
        while unvisited_cities:
            next_city_probabilities = np.array([probabilities[current_city][city] for city in unvisited_cities])
            next_city_probabilities /= next_city_probabilities.sum()
            next_city = np.random.choice(list(unvisited_cities), p=next_city_probabilities)
            self.tour.append(next_city)
            unvisited_cities.remove(next_city)
            current_city = next_city
        self.tour.append(self.end_city)
        self.tour_distance = self.calculate_tour_distance()
        self.total_value, self.total_weight = self.pack_items()
        self.tour_time = self.calculate_tour_time()

    def pack_items(self):
        best_value = 0
        best_weight = 0
        best_selection = []
        for _ in range(2):
            theta = random.uniform(0, 1)
            delta = random.uniform(0, 1 - theta)
            gamma = 1 - theta - delta
            item_distances = np.array([sum(distances[item_assignments[item], city] for city in self.tour) for item in range(num_items)])
            scores = (values ** theta) / (weights ** delta * item_distances ** gamma)
            sorted_items = np.argsort(scores)[::-1]
            current_value = 0
            current_weight = 0
            selected_items = []
            for item in sorted_items:
                if current_weight + weights[item] <= max_weight:
                    selected_items.append(item)
                    current_weight += weights[item]
                    current_value += values[item]
                else:
                    break
            if current_value > best_value:
                best_value = current_value
                best_weight = current_weight
                best_selection = selected_items
        return best_value, best_weight

class ACO:
    def __init__(self, num_ants, num_cities, start_city, end_city, pheromone_matrix, q=1.0, alpha=1.0, beta=2.0, rho=0.5):
        self.num_ants = num_ants
        self.num_cities = num_cities
        self.start_city = start_city
        self.end_city = end_city
        self.pheromone_matrix = pheromone_matrix
        self.q = q
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.ants = [Ant(num_cities, start_city, end_city) for _ in range(num_ants)]
        self.global_best_tour = None
        self.global_best_distance = float('inf')
        self.global_best_value = 0
        self.global_best_weight = 0

    def calculate_transition_probabilities(self):
        probabilities = np.zeros((self.num_cities, self.num_cities))
        for i in range(self.num_cities):
            for j in range(self.num_cities):
                if i != j:
                    numerator = (self.pheromone_matrix[i, j] ** self.q) * (1 / distances[i, j] ** self.alpha)
                    denominator = sum((self.pheromone_matrix[i, k] ** self.q) * (1 / distances[i, k] ** self.alpha) for k in range(self.num_cities) if k != i)
                    probabilities[i, j] = numerator / denominator
        return probabilities

    def update_pheromone_matrix(self):
        self.pheromone_matrix *= (1 - self.rho)
        for ant in self.ants:
            for i in range(len(ant.tour) - 1):
                self.pheromone_matrix[ant.tour[i], ant.tour[i + 1]] += self.rho / ant.tour_distance
    def calculate_tour_distance(self, tour=None):
      if tour is None:
          tour = self.tour
      tour_distance = 0.0
      for i in range(len(tour) - 1):
          tour_distance += distances[tour[i], tour[i + 1]]
      return tour_distance

    def local_search(self, ant):
        best_tour = ant.tour.copy()
        best_value = ant.total_value
        best_weight = ant.total_weight
        best_distance = ant.tour_distance
        improvement = True
        while improvement:
            improvement = False
            for i in range(1, self.num_cities - 2):
                for j in range(i + 1, self.num_cities - 1):
                    if j - i == 1:
                        continue
                    new_tour = ant.tour[:i] + ant.tour[i:j][::-1] + ant.tour[j:]
                    new_distance = self.calculate_tour_distance(new_tour)
                    if new_distance < best_distance:
                        ant.tour = new_tour
                        ant.total_value, ant.total_weight = ant.pack_items()
                        ant.tour_distance = new_distance
                        ant.tour_time = ant.calculate_tour_time()
                        if ant.total_value > best_value:
                            best_tour = ant.tour.copy()
                            best_value = ant.total_value
                            best_weight = ant.total_weight
                            best_distance = ant.tour_distance
                            improvement = True
                        break
                if improvement:
                    break
        ant.tour = best_tour
        ant.total_value = best_value
        ant.total_weight = best_weight
        ant.tour_distance = best_distance

    def solve(self):
        probabilities = self.calculate_transition_probabilities()
        for ant in self.ants:
            self.local_search(ant)
            ant.update_solution(probabilities)

        for ant in self.ants:
            if ant.total_value > self.global_best_value:
                self.global_best_distance = ant.tour_distance
                self.global_best_tour = ant.tour.copy()
                self.global_best_value = ant.total_value
                self.global_best_weight = ant.total_weight
        self.update_pheromone_matrix()
        return self.global_best_tour, self.global_best_distance, self.global_best_value, self.global_best_weight

    def plot_tour(self, tour, distance, value, weight, time_spent, frame_number):
        plt.figure(figsize=(8, 6))
        for i in range(self.num_cities):
            if i == self.start_city:
                plt.plot(cities_coordinates[i, 0], cities_coordinates[i, 1], 'o', markersize=5, color='green')
            elif i == self.end_city:
                plt.plot(cities_coordinates[i, 0], cities_coordinates[i, 1], 'o', markersize=5, color='red')
            else:
                plt.plot(cities_coordinates[i, 0], cities_coordinates[i, 1], 'o', markersize=5, color='blue')
            plt.text(cities_coordinates[i, 0], cities_coordinates[i, 1] + 0.02, i, ha='center', va='center', fontsize=8)
        for i in range(len(tour) - 1):
            plt.plot([cities_coordinates[tour[i], 0], cities_coordinates[tour[i + 1], 0]],
                     [cities_coordinates[tour[i], 1], cities_coordinates[tour[i + 1], 1]], 'b-', linewidth=2)
        plt.title(f'Distance: {distance:.2f}, Value: {value}, Weight: {weight}')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.savefig(f'frames/frame_{frame_number:04d}.png')
        plt.close()

q = 1.0
alpha = 1.0
beta = 2.0
rho = 0.5
start_city = 0
end_city = num_cities - 1
aco = ACO(num_ants, num_cities, start_city, end_city, pheromone_matrix, q, alpha, beta, rho)
best_distance = float('inf')
best_value = 0
best_tour = []
best_weight = 0
start_time = time.time()
if not os.path.exists('frames'):
    os.makedirs('frames')
frame_number = 0
while time.time() - start_time < time_limit:
    tour, distance, value, weight = aco.solve()
    if value > best_value:
        best_distance = distance
        best_value = value
        best_tour = tour
        best_weight = weight
    print(f"Best Distance: {best_distance}, Best Value: {best_value},Weight: {best_weight}")
    aco.plot_tour(best_tour, best_distance, best_value, best_weight, time.time() - start_time, frame_number)
    frame_number += 1
images = []
for file_name in sorted(os.listdir('frames')):
    if file_name.endswith('.png'):
        file_path = os.path.join('frames', file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave('tour.gif', images, fps=10)
print("Best Distance:", best_distance)
print("Best Value:", best_value)
print("Best Weight:", best_weight)
print("Best Solution:", best_tour)


Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.1504774806388, Best Value: 6801,Weight: 4001
Best Distance: 1182.15047

<ipython-input-31-457aa2d4c73f>:260: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Best Distance: 497.4816825584884
Best Value: 6861
Best Weight: 3961
Best Solution: [0, 21, 1, 15, 8, 48, 9, 29, 33, 49, 20, 28, 19, 2, 34, 35, 27, 30, 25, 7, 47, 22, 6, 42, 23, 5, 26, 45, 11, 46, 3, 16, 36, 43, 14, 44, 32, 38, 4, 37, 10, 31, 17, 13, 24, 12, 39, 41, 18, 40, 50]
